# Replay Script

This script will provide the ability to replay what has happened in the market, with the user being able to specify the time they wish to view. 

- Future iterations will allow for the simulation to be re-ran in real time, or possibly slowing down or speeding up the replay system.

In [13]:
# import libraries
%matplotlib agg
import time
import os
import io
import warnings
import mplfinance as mpf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from prettytable import PrettyTable
from ipywidgets import AppLayout


# Set the pandas option to opt into the future behavior, and suppress any SettingWithCopyWarning and FutureWarning messages
pd.set_option('future.no_silent_downcasting', True)
pd.options.mode.chained_assignment = None 
warnings.simplefilter(action='ignore', category=FutureWarning)

### WIDGET SETUP ####
file_name_input = widgets.Text(
    value="30mintest2.xlsx",            # default test value, to be removed to be blank once working
    desciprion='Enter file name e.g. Output.xlsx:',
    disabled=False,
    style={'description_width': 'initial'}
)
file_submit_button = widgets.ToggleButton(
    value = False,
    description='Click to submit.',
    disabled = False,
    icon='check'
)
file_submit = widgets.HTML()

time_int = widgets.IntText(
    value=0,
    min=0,
    step=1,
    description='Time after start (seconds):',
    style={'description_width': 'initial'}
    # layout=widgets.Layout(width='100px', height='50px', margin='auto')
)
time_slider_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=1,
    continuous_update=True
)

widgets.link((time_slider_widget, 'value'), (time_int, 'value'))
time_grid = widgets.GridspecLayout(2,1)
time_grid[0,0] = time_int
time_grid[1,0] = time_slider_widget


time_render_button = widgets.ToggleButton(
    value = False,
    description='Click to render.',
    disabled = False,
)
time_render = widgets.HTML()

# Ordering the widges into the header
top_grid = widgets.GridspecLayout(2,3)
top_grid[0,0] = file_name_input
top_grid[0,1] = file_submit_button
top_grid[0,2] = file_submit
top_grid[1,0] = time_grid
top_grid[1,1] = time_render_button
top_grid[1,2] = time_render

b_orderbook_title = widgets.HTML(value="""
                                 <html>
                                    <h3>
                                        <i>Buy Orderbook</i>
                                    </h3>
                                </html> """)
s_orderbook_title = widgets.HTML(value="""
                                 <html>
                                    <h3>
                                        <i>Sell Orderbook</i>
                                    </h3>
                                </html> """)
b_orderbook_widget = widgets.HTML()
s_orderbook_widget = widgets.HTML()
chart_widget = widgets.Image(format='png')

centre_grid=widgets.GridspecLayout(2,3)
centre_grid[0,0] = b_orderbook_title
centre_grid[1,0] = b_orderbook_widget
centre_grid[1,1] = chart_widget
centre_grid[0,2] = s_orderbook_title
centre_grid[1,2] = s_orderbook_widget
centre_grid.layout.grid_template_rows = '100px auto'


### Functions to handle and update the front end ###
# Creating an "output" folder which is used to read in the file, and creating the file input output
def file_read(*args): 
    global orderbook_log, transaction_log, start_time
    
    try:
        output_path = 'output'
        file_name = file_name_input.value
        final_output_path = os.path.join(output_path, file_name)

        # reading the input file, passing the first column as the index
        orderbook_log = pd.read_excel(final_output_path, sheet_name='Orderbook_Log', index_col=0)
        transaction_log = pd.read_excel(final_output_path, sheet_name='Transaction_Log', index_col=0)

        # update front end to be success
        file_submit_button.button_style = 'success'
        file_submit.value = """
        <html>
            <body>
                <b>
                {file_name} was successfully read, and ready to be visualised!
                </b>
            </body>
        </html>
        """.format(file_name= file_name)

        # update the ranges for the time slider
        start_time = orderbook_log.iloc[0]["Timestamp"].timestamp()
        end_time = orderbook_log.iloc[-1]["Timestamp"].timestamp()
        end_delta = end_time - start_time
        time_slider_widget.max = end_delta

    except:
        # update front end to be failure
        file_submit_button.button_style = 'danger'
        file_submit.value = """
        <html>
            <body>
                <b>
                {file_name} could not be read. Please check the file name.
                </b>
            </body>
        </html>
        """.format(file_name= file_name)
file_submit_button.observe(file_read, 'value')


# function that handles the creation of orderbook vis
def orderbook_vis (orderbook, side, html_flag):
    agg_orderbook = pd.DataFrame(columns=["Price", "Quantity"])
    agg_orderbook = orderbook.groupby("Price").agg({"Quantity" : 'sum'}).reset_index()
    agg_orderbook.columns = ['Price', 'Total Quantity']
    
    if html_flag == True:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total Quantity']
        if side == "b":
            agg_orderbook.sort_values(by=['Price'], ascending=False, inplace=True) 
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total Quantity']])
        vis = vis.get_html_string()
    elif html_flag == False:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total Quantity']
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total Quantity']])
    return vis

# function to update the buy_orderbook based on the time selected by the user
def update_orderbook (time, side):
    global orderbook_log
    orderbook = pd.DataFrame(columns=["Order_ID", "Trader_ID", "Timestamp", "Quantity", "Price"])

    if side == "Buy":
        for index, order in orderbook_log.iterrows():
            if (order["Side"] == "Buy" and order["Status"] == "Open") and order["Timestamp"] <= time:
                new_order = {"Order_ID": order["Order_ID"], "Trader_ID" : order["Trader_ID"], "Timestamp" : order["Timestamp"], "Quantity" : order["Quantity"], "Price" : order["Price"]}
                to_orderbook = pd.Series(new_order)
                orderbook = pd.concat([orderbook, to_orderbook.to_frame().T], ignore_index=True)
        orderbook.sort_values(by=["Price", "Timestamp"], ascending=[False, True], inplace=True)
        orderbook_widget_html = orderbook_vis(orderbook, "b", html_flag=True)
    elif side == "Sell":
        for index, order in orderbook_log.iterrows():
            if (order["Side"] == "Buy" and order["Status"] == "Open") and order["Timestamp"] <= time:
                new_order = {"Order_ID": order["Order_ID"], "Trader_ID" : order["Trader_ID"], "Timestamp" : order["Timestamp"], "Quantity" : order["Quantity"], "Price" : order["Price"]}
                to_orderbook = pd.Series(new_order)
                orderbook = pd.concat([orderbook, to_orderbook.to_frame().T], ignore_index=True)
        orderbook.sort_values(by=["Price", "Timestamp"], ascending=[True, True], inplace=True)
        orderbook_widget_html = orderbook_vis(orderbook, "s", html_flag=True)
    return orderbook_widget_html

# fuction to update the candlestick chart
def update_transaction (time, time_elapsed):
    global transaction_log

    # pulling the transactions that fall within the time window, and ordering them into candles
    vis_transaction = pd.DataFrame(columns=["Timestamp", "Buy_Side_Order_ID", "Buy_Side_Trader_ID", "Sell_Side_Order_ID", "Sell_Side_Trader_ID", "Quantity", "Price", "Aggressor", "Aggressor_ID"])  
    for index, transaction in transaction_log.iterrows():
        if transaction["Timestamp"] <= time:
            vis_transaction = pd.concat([vis_transaction, transaction.to_frame().T], ignore_index=True)
    vis_transaction["Timestamp"] = pd.to_datetime(vis_transaction["Timestamp"])
    vis_transaction.sort_values(by=["Timestamp"], ascending=[True], inplace=True)

    if time_elapsed < 180:
        candle_length = 15 
    elif time_elapsed >= 180 and time_elapsed < 300:
        candle_length = 30
    elif time_elapsed >= 300 and time_elapsed < 900:
        candle_length = 60
    elif time_elapsed >= 900:
        candle_length = 120

    str_candle_length = "".join((str(candle_length),'s'))
    vis_transaction["timestamp"] = vis_transaction['Timestamp'].dt.floor(str_candle_length)
    vis_transaction = vis_transaction.groupby("timestamp").agg({
        "Price" : ['first', 'max', 'min','last'],
        "Quantity" : 'sum'
    })
    vis_transaction.columns = ['open', 'high', 'low', 'close', 'volume']
    columns_to_convert = ['open', 'high', 'low', 'close']
    vis_transaction[columns_to_convert] = vis_transaction[columns_to_convert].round(2).astype(float)
    vis_transaction["volume"] = vis_transaction["volume"].astype(int)

    # chart settings
    chart_colors = mpf.make_marketcolors(
        up='green',
        down='red'
    )
    chart_style = mpf.make_mpf_style(
        marketcolors = chart_colors,
        gridcolor='lightgray',        
        gridstyle='--'
    )

    fig, ax = mpf.plot(vis_transaction, type='candle', style=chart_style, volume=True, returnfig=True)   
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    chart_widget_html = buf.getvalue()

    return chart_widget_html

# function to render a specific point in time
def render (*args):
    global start_time
    if time_render_button.value==True:
        time_value = time_int.value
        time_elapsed = (start_time + time_value) - 3600
        datetime_point = dt.datetime.fromtimestamp(time_elapsed)
        print(datetime_point)
        b_orderbook_widget.value = update_orderbook(datetime_point, "Buy")
        s_orderbook_widget.value = update_orderbook(datetime_point, "Sell")
        chart_widget.value = update_transaction(datetime_point, time_elapsed)
        time_render.value = """
            <html>
                <body>
                    <b>
                    Successfully rendered. Results are displayed below:
                    </b>
                </body>
            </html>
            """
        
        # reset the button
        time.sleep(1)
        time_render_button.value=False

time_render_button.observe(render, 'value')

AppLayout(header=top_grid,
          center=centre_grid)



AppLayout(children=(GridspecLayout(children=(Text(value='30mintest2.xlsx', layout=Layout(grid_area='widget001'…

2024-07-25 14:03:45.967000
2024-07-25 14:05:35.967000
2024-07-25 14:12:22.967000
